<a href="https://colab.research.google.com/github/World4AI/World4AI/blob/main/website/src/notebooks/convolutional_neural_networks/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

import os
import pathlib
import zipfile
import shutil

In [2]:
!mkdir ~/.kaggle

In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -p /content/datasets  -c dogs-vs-cats

 98% 796M/812M [00:04<00:00, 158MB/s]
100% 812M/812M [00:04<00:00, 178MB/s]


In [6]:
root = pathlib.Path('/content/datasets')

In [7]:
if not os.path.exists(root / 'dogs-vs-cats'):
  with zipfile.ZipFile(root / 'dogs-vs-cats.zip', 'r') as zip_ref:
      zip_ref.extractall(root / 'dogs-vs-cats')
      
  with zipfile.ZipFile(root / 'dogs-vs-cats/train.zip', 'r') as zip_ref:
      zip_ref.extractall(root / 'dogs-vs-cats')


In [8]:
original_path = root / 'dogs-vs-cats'
new_path = root / 'dogs_vs_cats_prepared'

In [9]:
# prepare list of files
file_names = os.listdir(original_path / 'train')
test_file_names = file_names[0:5000]
val_file_names = file_names[5000:10000]
train_file_names = file_names[10000:]

In [10]:
# prepare dirs
for directory in ["train", "test", "val"]:
    for category in ["cat", "dog"]:
        new_dir = new_path / directory / category
        if os.path.exists(new_dir):
            shutil.rmtree(new_dir)
        os.makedirs(new_dir)

In [11]:
def copy_files(old_dir, new_dir, files_list):
    for idx, filename in enumerate(files_list):
        src = old_dir / filename
        if filename.startswith('cat'):
            dst = new_dir / 'cat' / filename
        elif filename.startswith('dog'):
            dst = new_dir / 'dog' / filename
        else:
            continue
            
        shutil.copyfile(src=src, dst=dst)

In [12]:
copy_files(old_dir=original_path / "train", new_dir=new_path / 'train', files_list=train_file_names)
copy_files(old_dir=original_path / "train", new_dir=new_path / 'val', files_list=val_file_names)
copy_files(old_dir=original_path / "train", new_dir=new_path / 'test', files_list=test_file_names)

In [13]:
root = new_path

In [38]:
train_transform = T.Compose([T.Resize((256, 256)), 
                             T.RandomCrop(size=(224, 224)),
                             T.ToTensor()])

test_transform = T.Compose([T.Resize((224, 224)), 
                                T.ToTensor()])

In [39]:
train_dataset = ImageFolder(root=root / 'train', transform=train_transform)
val_dataset = ImageFolder(root=root / 'val', transform=test_transform)
test_dataset = ImageFolder(root=root / 'test', transform=test_transform)

In [40]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE=128

In [41]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=False)

In [20]:
# standard convolution for the first layer
def conv_std(in_channels, out_channels, stride):
  return nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
      nn.BatchNorm2d(num_features=out_channels),
      nn.ReLU(inplace=True)) 

In [21]:
# depthwise separable convolution
def conv_ds(in_channels, out_channels, stride):
    return nn.Sequential(
        nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
        nn.BatchNorm2d(num_features=in_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(num_features=out_channels),
        nn.ReLU(inplace=True)     
      ) 

In [22]:
class Model(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        conv_std(3, 32, 2),
        conv_ds(32, 64, 1),
        conv_ds(64, 128, 2),
        conv_ds(128, 128, 1),
        conv_ds(128, 256, 2), 
        conv_ds(256, 256, 1),        
        conv_ds(256, 512, 2), 
        conv_ds(512, 512, 1),   
        conv_ds(512, 512, 1),   
        conv_ds(512, 512, 1),   
        conv_ds(512, 512, 1),   
        conv_ds(512, 512, 1), 
        conv_ds(512, 1024, 2),  
        # bug in the paper?
        conv_ds(1024, 1024, 1),  
        nn.AdaptiveAvgPool2d(1),
        nn.Flatten(),
        nn.Linear(1024, 1)
    )

  def forward(self, x):
    return self.layers(x)

In [23]:
def track_performance(dataloader, model, criterion):
    # switch to evaluation mode
    model.eval()
    num_samples = 0
    num_correct = 0
    loss_sum = 0
    
    # no need to calculate gradients
    with torch.inference_mode():
        for batch_idx, (features, labels) in enumerate(dataloader):
            features = features.to(DEVICE)
            labels = labels.to(DEVICE).view(-1, 1).float()
            logits = model(features)
            probs = torch.sigmoid(logits)
                        
            predictions = (probs > 0.5).float()
            num_correct += (predictions == labels).sum().item()
            
            loss = criterion(logits, labels)
            loss_sum += loss.cpu().item()
            num_samples += len(features)
    
    # we return the average loss and the accuracy
    return loss_sum/num_samples, num_correct/num_samples

In [24]:
def train(num_epochs, train_dataloader, val_dataloader, model, criterion, optimizer, scheduler=None):
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    
    model.to(DEVICE)
    
    for epoch in range(num_epochs):
        for batch_idx, (features, labels) in enumerate(train_dataloader):
            model.train()
            features = features.to(DEVICE)
            labels = labels.to(DEVICE).view(-1, 1).float()
            
            # Empty the gradients
            optimizer.zero_grad()
            
            # Forward Pass
            logits = model(features)
            
            # Calculate Loss
            loss = criterion(logits, labels)
            
            # Backward Pass
            loss.backward()
            
            # Gradient Descent
            optimizer.step()
            
        train_loss, train_acc = track_performance(train_dataloader, model, criterion)
        val_loss, val_acc = track_performance(val_dataloader, model, criterion)

        if scheduler:
          scheduler.step(val_loss)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        print(f'Epoch: {epoch+1:>2}/{num_epochs} | Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}')
    return history            
            

In [25]:
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       patience=2,
                                                       verbose=True)
criterion = nn.BCEWithLogitsLoss()

In [26]:
history = train(10, train_dataloader, val_dataloader, model, criterion, optimizer, scheduler)

Epoch:  1/10 | Train Loss: 0.00451 | Val Loss: 0.00478 | Train Acc: 0.700 | Val Acc: 0.694
Epoch:  2/10 | Train Loss: 0.00392 | Val Loss: 0.00419 | Train Acc: 0.759 | Val Acc: 0.758
Epoch:  3/10 | Train Loss: 0.00375 | Val Loss: 0.00422 | Train Acc: 0.765 | Val Acc: 0.744
Epoch:  4/10 | Train Loss: 0.00474 | Val Loss: 0.00502 | Train Acc: 0.735 | Val Acc: 0.715
Epoch:  5/10 | Train Loss: 0.00290 | Val Loss: 0.00327 | Train Acc: 0.833 | Val Acc: 0.813
Epoch:  6/10 | Train Loss: 0.00267 | Val Loss: 0.00327 | Train Acc: 0.845 | Val Acc: 0.808
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch:  7/10 | Train Loss: 0.00241 | Val Loss: 0.00298 | Train Acc: 0.872 | Val Acc: 0.844
Epoch:  8/10 | Train Loss: 0.00168 | Val Loss: 0.00234 | Train Acc: 0.913 | Val Acc: 0.872
Epoch:  9/10 | Train Loss: 0.00158 | Val Loss: 0.00221 | Train Acc: 0.917 | Val Acc: 0.883
Epoch 00010: reducing learning rate of group 0 to 1.0000e-05.
Epoch: 10/10 | Train Loss: 0.00149 | Val Loss: 0.00219 | 

In [27]:
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

In [42]:
model = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V2, progress=False)

In [43]:
for param in model.parameters():
  param.requires_grad = False

In [44]:
model.classifier[3] = nn.Linear(in_features=1280, out_features=1)

In [45]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       patience=2,
                                                       verbose=True)
criterion = nn.BCEWithLogitsLoss()

In [46]:
history = train(10, train_dataloader, val_dataloader, model, criterion, optimizer, scheduler)

Epoch:  1/10 | Train Loss: 0.00124 | Val Loss: 0.00136 | Train Acc: 0.970 | Val Acc: 0.967
Epoch:  2/10 | Train Loss: 0.00095 | Val Loss: 0.00107 | Train Acc: 0.971 | Val Acc: 0.967
Epoch:  3/10 | Train Loss: 0.00075 | Val Loss: 0.00087 | Train Acc: 0.974 | Val Acc: 0.969
Epoch 00004: reducing learning rate of group 0 to 1.0000e-04.
Epoch:  4/10 | Train Loss: 0.00067 | Val Loss: 0.00077 | Train Acc: 0.973 | Val Acc: 0.971
Epoch:  5/10 | Train Loss: 0.00066 | Val Loss: 0.00076 | Train Acc: 0.975 | Val Acc: 0.972
Epoch:  6/10 | Train Loss: 0.00066 | Val Loss: 0.00075 | Train Acc: 0.973 | Val Acc: 0.972
Epoch 00007: reducing learning rate of group 0 to 1.0000e-05.
Epoch:  7/10 | Train Loss: 0.00066 | Val Loss: 0.00075 | Train Acc: 0.973 | Val Acc: 0.972
Epoch:  8/10 | Train Loss: 0.00065 | Val Loss: 0.00075 | Train Acc: 0.975 | Val Acc: 0.972
Epoch:  9/10 | Train Loss: 0.00066 | Val Loss: 0.00074 | Train Acc: 0.974 | Val Acc: 0.972
Epoch 00010: reducing learning rate of group 0 to 1.0000e